# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer,VectorAssembler, Normalizer,StandardScaler,MinMaxScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [5]:
df.show()

+--------------------+---+--------------------+--------------------+----------------+
|                Body| Id|                Tags|               Title|          oneTag|
+--------------------+---+--------------------+--------------------+----------------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|             php|
|<p>In my favorite...|  2|             firefox|How can I prevent...|         firefox|
|<p>I am import ma...|  3|r matlab machine-...|R Error Invalid t...|               r|
|<p>This is probab...|  4|     c# url encoding|How do I replace ...|              c#|
|<pre><code>functi...|  5|php api file-get-...|How to modify who...|             php|
|<p>I am using a m...|  6|proxy active-dire...|setting proxy in ...|active-directory|
|<p>My image is ca...|  7|           core-plot|How to draw barpl...|           other|
|<p>I've decided t...|  8|c# asp.net window...|How to fetch an X...|              c#|
|<p>Do you know of...|  9|.net javascript c...|.NET li

### Build Body Length Feature

In [6]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [7]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [8]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [9]:
df.createOrReplaceTempView("t1")
# TODO: write your code to answer question 1
spark.sql('''
            SELECT BodyLength
            FROM t1
            WHERE id=1112''').show()

+----------+
|BodyLength|
+----------+
|        63|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [10]:
spark.udf.register("textlength",lambda x: len(x),IntegerType())

<function __main__.<lambda>(x)>

In [11]:
# TODO: write your code to answer question 2
df=spark.sql('''
            SELECT *, CONCAT(title, body) titlebody
            FROM t1
            ''')

In [12]:
regextokenizer=RegexTokenizer(inputCol="titlebody",outputCol="titlebodys",pattern="\\W")
df=regextokenizer.transform(df)

In [13]:
df.createOrReplaceTempView("t1")
spark.sql('''
            SELECT textlength(titlebodys)
            FROM T1
            WHERE ID=5123''').show()

+----------------------+
|textlength(titlebodys)|
+----------------------+
|                   135|
+----------------------+



In [14]:
df=spark.sql('''
            SELECT *, textlength(titlebodys) length
            FROM T1''')

In [15]:
df.take(1)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [16]:
# TODO: write your code to create this vector
assembler=VectorAssembler(inputCols=['length'], outputCol="vector")
df=assembler.transform(df)
df.take(1)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [17]:
# TODO: write your code to answer question 3
scaler=Normalizer(inputCol="vector",outputCol="ScaledNumFeatures")
df=scaler.transform(df)

In [18]:
df.createOrReplaceTempView('t1')
spark.sql('''
        SELECT ScaledNumFeatures
        FROM t1
        WHERE ID=512''').show()

+-----------------+
|ScaledNumFeatures|
+-----------------+
|            [1.0]|
+-----------------+



# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [19]:
# TODO: write your code to answer question 4
scaler2=StandardScaler(inputCol="vector",outputCol="ScaledNumFeatures2",withStd=True, withMean=True)
scalermodel=scaler2.fit(df)
df=scalermodel.transform(df)

In [20]:
df.createOrReplaceTempView('t1')
spark.sql('''
        SELECT ScaledNumFeatures2
        FROM t1
        WHERE id=512''').show()

+--------------------+
|  ScaledNumFeatures2|
+--------------------+
|[-0.6417314460998...|
+--------------------+



# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [21]:
# TODO: write your code to answer question 5
scaler3=MinMaxScaler(inputCol="vector",outputCol="scaledFeatures")
model=scaler3.fit(df)
df=model.transform(df)

In [22]:
df.createOrReplaceTempView('t1')
spark.sql('''
SELECT scaledFeatures
FROM T1
WHERE id=512''').show()

+--------------------+
|      scaledFeatures|
+--------------------+
|[0.00624833820792...|
+--------------------+

